# Workshop on PDMP

In this worshop we will 

- Example Random walk vs lifted random walk. 
- Reproduce the plot 
- Plot the autocorrelation function. 

- Simulate data from the linear regression (1 dimensional case  case)
- Sample from a quadratic potential 
- Plot traces, plot empirical distribution (histogram) 
### Random Walk
Simulate and save the trajectory of a random walk on a lattice $\{1,2,\dots,N\}$. The random walk move from $x 
\to x \pm 1$ with probability $1/2$. 

In [ ]:
# template for random walk 

In [ ]:
# SOLUTIONS WITH MARKS 

### Lifted Random Walk
Simulate and save....

In [ ]:
# Template lifted random walk


In [ ]:
# SOLUTIONS WITH MARKS 

# Plotting autocorrelation and traces
- reproduce the trace plot 
- Compute and compare autocorrelation of rw and lrw


In [ ]:
# template for plots 
# use numpy.correlate

In [ ]:
# SOLUTIONS WITH MARKS 

Simulate data from linear regression with one covariate
- simulate $N=100$ observations:
- $x_i \sim \mathcal{N}(0,1)$
- $y_i \sim \mathcal{N}(x_i\beta, \sigma^2)$, $\sigma = 1.0, \beta = 0.5$

In [ ]:
# template for simulating data 
# use numpy.random.normal
# store y = (y_1,y_2,...,y_N) as a numpy.array 
#  ...

In [ ]:
# SOLUTIONS WITH MARKS 

# Sample from the quadratic potential 
- In this case $\lambda(t) = max(0, a + b*t)$
- simulate $\tau$ such that $P(\tau > t) = \exp(- \int_0^t \lambda(s) ds)$


In [ ]:
# provide the function 
# \lambda(t) = max(0, a + b*t)
#  simulate \tau such that P(\tau > t) = \exp(- \int_0^t \lambda(s) ds)
import math
def poisson_time(a, b, u):
    if b > 0:
        if a < 0:
            return math.sqrt(-math.log(u)*2.0/b) - a/b
        else:  # a > 0
            return math.sqrt((a/b)**2 - math.log(u)*2.0/b) - a/b
    elif b == 0:
        if a > 0:
            return -math.log(u)/a
        else:
            return float('inf')
    else:  # b < 0
        if a <= 0:
            return float('inf')
        elif -math.log(u) <= -a**2/b + a**2/(2*b):
            return -math.sqrt((a/b)**2 - math.log(u)*2.0/b) - a/b
        else:
            return float('inf')

# Example usage
result = poisson_time(2, 3, 0.5)
print(result)

### Derive analyitically posterior 
- Set a Gaussian proir
- Derive the Gaussian posterior 
- What is its mean?
image.png


### Simulate PDMP and plot trace and compute empirical mean
- Fill in the gaps in the tamplate
- Figure how to plot the trace
- Figure how to derive the mean from the skeleton points 

![image](procedure_zz.png)